In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
%%shell
pip uninstall -y keras
pip install keras==2.2.4

Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1


# 물체 탐지(Object detection) - YOLO Keras

DarkNet의 모델을 Keras로 변환하여 물체 탐지 실행.

https://github.com/qqwweee/keras-yolo3


## 프로젝트 다운로드


In [2]:
%cd /content

!rm -rf keras-yolo3

!git clone https://github.com/qqwweee/keras-yolo3


/content
Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 672.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


## YOLO3 모델 다운로드


In [3]:
%cd /content/keras-yolo3
!wget wget https://pjreddie.com/media/files/yolov3.weights

/content/keras-yolo3
--2020-08-24 03:38:12--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2020-08-24 03:38:12--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   486KB/s    in 7m 46s  

2020-08-24 03:46:00 (519 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]

FINISHED --2020-08-24 03:46:00--
Total wall clock time: 7m 48s
Downloaded: 1 files, 237M in 7m 46s (519 KB/s)


## 모델 파일 변환

DarkNet의 모델파일을 Keras에서 사용할 수 있는 모델 파일로 변환한다.


- convert.py : 변환 실행 파일
- yolov3.cfg : Darknet에서 사용하는 모델 구조 정의 파일
- yolov3.weight : Darknet으로 학습된 모델 파일

실행 결과로 다음의 h5 파일로 변환된다. 
- yolov3.h5 : 변환된 모델 파일


In [5]:
!head -40 yolov3.cfg 

[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1


In [6]:
# 실행하기전 기존것이 있다면 삭제
!rm -rf yolov3.h5

# 실행 
%run convert.py yolov3.cfg yolov3.weights yolov3.h5

Using TensorFlow backend.


Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)









Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_1
Parsing section convolutional_7
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_2
Parsing section convolutional_9
conv2d bn leaky (3, 3, 128, 256)
Parsing section convolutional_10
conv2d bn leaky (1, 1, 256, 128)
Parsing section convolutional_11
conv2d bn leaky (3, 3, 128, 256)
Parsing section short

In [7]:
!ls -al 

total 485184
drwxr-xr-x 6 root root      4096 Aug 24 03:54 .
drwxr-xr-x 1 root root      4096 Aug 24 03:38 ..
-rw-r--r-- 1 root root      1323 Aug 24 03:38 coco_annotation.py
-rw-r--r-- 1 root root     10093 Aug 24 03:38 convert.py
-rw-r--r-- 1 root root      5707 Aug 24 03:38 darknet53.cfg
drwxr-xr-x 2 root root      4096 Aug 24 03:38 font
drwxr-xr-x 8 root root      4096 Aug 24 03:38 .git
-rw-r--r-- 1 root root      1201 Aug 24 03:38 .gitignore
-rw-r--r-- 1 root root      3482 Aug 24 03:38 kmeans.py
-rw-r--r-- 1 root root      1064 Aug 24 03:38 LICENSE
drwxr-xr-x 2 root root      4096 Aug 24 03:38 model_data
-rw-r--r-- 1 root root      4077 Aug 24 03:38 README.md
-rw-r--r-- 1 root root     10686 Aug 24 03:38 train_bottleneck.py
-rw-r--r-- 1 root root      8332 Aug 24 03:38 train.py
-rw-r--r-- 1 root root      1416 Aug 24 03:38 voc_annotation.py
drwxr-xr-x 2 root root      4096 Aug 24 03:38 yolo3
-rw-r--r-- 1 root root      8540 Aug 24 03:38 yolo.py
-rw-r--r-- 1 root root      8342 Au

# 커스텀 데이터 학습


## 커스터 데이터 다운로드

standford dogs 데이터셋 중 10개 클래스만 추린 데이타

In [ ]:
%cd /content/keras-yolo3

!rm -rf dogs.tar.gz

!wget https://github.com/dhrim/mnd_2020/raw/master/material/deep_learning/dogs.tar.gz


/content/keras-yolo3
--2020-08-24 03:32:14--  https://github.com/dhrim/mnd_2020/raw/master/material/deep_learning/dogs.tar.gz
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/mnd_2020/master/material/deep_learning/dogs.tar.gz [following]
--2020-08-24 03:32:15--  https://raw.githubusercontent.com/dhrim/mnd_2020/master/material/deep_learning/dogs.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62885106 (60M) [application/octet-stream]
Saving to: ‘dogs.tar.gz’

dogs.tar.gz         100%[===================>]  59.97M   192MB/s    in 0.3s    

2020-08-24 03:32:25 (192 MB/s) - ‘dogs.tar.gz’ save

In [ ]:
%cd /content/keras-yolo3

!rm -rf dogs

!tar xfz dogs.tar.gz

!ls -al dogs | head -10

/content/keras-yolo3
total 128
drwxr-xr-x 12  501 staff  4096 Jul 23  2019 .
drwxr-xr-x  8 root root   4096 Aug 24 03:32 ..
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02085620-Chihuahua
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02085782-Japanese_spaniel
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02085936-Maltese_dog
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02086079-Pekinese
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02086240-Shih-Tzu
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02086646-Blenheim_spaniel
drwxr-xr-x  2  501 staff 12288 Aug 24 03:32 n02086910-papillon


## 데이터 준비

2개의 파일

- 클래스 파일
- 레이블링 파일

그리고 이미지 파일들

In [ ]:
!wget https://github.com/dhrim/mnd_2020/raw/master/material/deep_learning/dogs_classes.txt
!wget https://github.com/dhrim/mnd_2020/raw/master/material/deep_learning/dogs_annotations.txt
  

--2020-08-24 03:32:32--  https://github.com/dhrim/mnd_2020/raw/master/material/deep_learning/dogs_classes.txt
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/mnd_2020/master/material/deep_learning/dogs_classes.txt [following]
--2020-08-24 03:32:32--  https://raw.githubusercontent.com/dhrim/mnd_2020/master/material/deep_learning/dogs_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228 [text/plain]
Saving to: ‘dogs_classes.txt’

dogs_classes.txt    100%[===================>]     228  --.-KB/s    in 0s      

2020-08-24 03:32:33 (14.2 MB/s) - ‘dogs_classes.txt’ saved [228/228]

--2020-

In [ ]:
!cat dogs_classes.txt

n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound


In [ ]:
!wc dogs_annotations.txt
!head -10 dogs_annotations.txt

  1918   3934 122615 dogs_annotations.txt
dogs/n02086240-Shih-Tzu/n02086240_8825.jpg 97,46,436,464,0
dogs/n02086240-Shih-Tzu/n02086240_4608.jpg 0,100,496,371,0
dogs/n02086240-Shih-Tzu/n02086240_1779.jpg 213,114,361,331,0
dogs/n02086240-Shih-Tzu/n02086240_5352.jpg 0,57,375,491,0
dogs/n02086240-Shih-Tzu/n02086240_9098.jpg 156,9,348,375,0
dogs/n02086240-Shih-Tzu/n02086240_963.jpg 0,8,325,492,0
dogs/n02086240-Shih-Tzu/n02086240_5541.jpg 63,105,345,284,0
dogs/n02086240-Shih-Tzu/n02086240_7100.jpg 6,23,372,456,0
dogs/n02086240-Shih-Tzu/n02086240_4629.jpg 32,57,451,373,0
dogs/n02086240-Shih-Tzu/n02086240_1958.jpg 43,16,375,495,0


## 학습 실행

학습에 필요한 파일은 3개이다.

- 클래스 파일
- 레이블링 파일
- 기반 학습 모델




그런데 2019/08/07 현재 이 파일 이름들은 train.py안에 하드코딩 되어있다.
- 클래스 파일 : model_data/voc_classes.txt
- 레이블링 파일 : train.txt
- 기반 학습 모델 : model_data/yolo_weights.h5

하드 코딩된 바 데로 이름을 변경하고 실행한다.



In [ ]:
%%shell
cp dogs_classes.txt model_data/voc_classes.txt
cp dogs_annotations.txt train.txt
cp yolov3.h5 model_data/yolo_weights.h5


In [ ]:
!python3 train.py

Using TensorFlow backend.








2020-08-24 03:32:46.396611: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-08-24 03:32:46.401427: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000185000 Hz
2020-08-24 03:32:46.401627: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x22e6bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-08-24 03:32:46.401656: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-08-24 03:32:46.403641: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-24 03:32:46.503963: W tensorflow/compiler/xla/service/platform_util.cc:256] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDev

In [ ]:
from IPython.display import display 
from PIL import Image
from yolo import YOLO

def do_object_detection(file, model_path, class_path):
  
  yolo = YOLO(model_path=model_path, classes_path=class_path)

  image = Image.open(file)
  result_image = yolo.detect_image(image)

  display(result_image)


In [ ]:
do_object_detection('dogs/n02086240-Shih-Tzu/n02086240_1011.jpg', 'model_data/yolo_weights.h5', 'model_data/voc_classes.txt')

AssertionError: ignored